In [704]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
from datetime import *

past_pred = pd.read_csv("past_game_predictions_12_15_merge.csv")
future_pred = pd.read_csv("future_game_predictions_12_15.csv")
all_games = pd.read_csv("allgames_12_15.csv")

past_2017 = pd.read_csv('past_2017.csv').dropna() #dropna to get rid of AFC vs NFC game
past_2018 = pd.read_csv('past_2018.csv').dropna() #dropna to get rid of AFC vs NFC game
past_2019 = pd.read_csv('past_2019_12_15.csv')
future_2019 = pd.read_csv('future_12_15.csv')


all_games['Week'] = all_games['Week'].replace(['WC', 'DP', 'CC', 'SB'], [18, 19, 20, 21])
all_games['Week'] = all_games['Week'].astype(int)

past_2019.append(future_2019)
past_pred = past_pred[['GameNum',
 'Predicted_Spread']]

past_pred = past_pred.merge(all_games[all_games['H/A']=='H'][['GameNum','PF','PA']], left_on='GameNum', right_on='GameNum')

past_pred['Game_Spread'] = past_pred['PA'].astype(int) - past_pred['PF'].astype(int)
past_pred.drop(['PF','PA'], axis=1, inplace=True)

past_pred.sort_values('GameNum')

,GameNum,Predicted_Spread,Game_Spread
64129,185,3.0,-12
64044,185,-1.0,-12
64043,185,2.0,-12
64042,185,0.0,-12
64041,185,-2.0,-12
64040,185,1.0,-12
64039,185,-1.0,-12
64038,185,2.0,-12
64037,185,2.0,-12
64036,185,3.0,-12


In [705]:
current_week = all_games[all_games['Season']==2019]['Week'].max() # get rid of -1 when predicting future

future_2019['Week'] = current_week
future_2019['Season'] = 2019

# Derive Week and season of lines to create composite key with all_games

def getWeek2019(s):
        new_date = date(int(s[:4]), int(s[5:7]), int(s[8:10]))
        for i in range(1,current_week+1):
            if (new_date < date(2019,9,3) + timedelta(7*i)):
                return i
            
past_2019['Week'] = past_2019['Date'].apply(getWeek2019)
past_2019['Season'] = 2019

def getWeek2018(s):
        new_date = date(int(s[:4]), int(s[5:7]), int(s[8:10]))
        for i in range(1,22):
            if (new_date < date(2018,9,4) + timedelta(7*i)):
                return i
        return 21
            
past_2018['Week'] = past_2018['Date'].apply(getWeek2018)
past_2018['Season'] = 2018

def getWeek2017(s):
        new_date = date(int(s[:4]), int(s[5:7]), int(s[8:10]))
        for i in range(1,22):
            if (new_date < date(2017,9,5) + timedelta(7*i)):
                return i
        return 21
            
past_2017['Week'] = past_2017['Date'].apply(getWeek2017)
past_2017['Season'] = 2017

past_2019 = past_2019[past_2019['Week'] < current_week]

lines = past_2017.append(past_2018).append(past_2019)

keys = all_games[(all_games['H/A']=='H')][['GameNum','Opposing Team','Week','Team','Season']]

lines = lines.merge(keys,left_on=['Week','Home', 'Away','Season'],right_on=['Week','Team', 'Opposing Team','Season'])
lines = lines.drop(columns=['Unnamed: 0','Date','Opposing Team','Team','Week','Season'])
lines.drop('Unnamed: 0.1',axis=1,inplace=True)
lines['GameNum'].max()

3679

In [706]:
# past_pred = past_pred[['GameNum',
#  'Predicted_Spread',
#  'Game_Spread',
#  'Away',
#  'Away_Spread',
#  'Away_Payout',
#  'Home',
#  'Home_Spread',
#  'Home_Payout',
#  'Vegas_Line',
#  'Win',]]

past_pred = past_pred.merge(lines, left_on='GameNum', right_on='GameNum')
past_pred

,GameNum,Predicted_Spread,Game_Spread,Away,Away_Payout,Away_Spread,Home,Home_Payout,Home_Spread
0,3195,-3.0,-30,Pittsburgh Steelers,-107.0,5.5,New England Patriots,-103.0,-5.5
1,3195,-3.0,-30,Pittsburgh Steelers,-107.0,5.5,New England Patriots,-103.0,-5.5
2,3195,-5.0,-30,Pittsburgh Steelers,-107.0,5.5,New England Patriots,-103.0,-5.5
3,3195,-3.0,-30,Pittsburgh Steelers,-107.0,5.5,New England Patriots,-103.0,-5.5
4,3195,-6.0,-30,Pittsburgh Steelers,-107.0,5.5,New England Patriots,-103.0,-5.5
5,3195,-2.0,-30,Pittsburgh Steelers,-107.0,5.5,New England Patriots,-103.0,-5.5
6,3195,-3.0,-30,Pittsburgh Steelers,-107.0,5.5,New England Patriots,-103.0,-5.5
7,3195,-2.0,-30,Pittsburgh Steelers,-107.0,5.5,New England Patriots,-103.0,-5.5
8,3195,-7.0,-30,Pittsburgh Steelers,-107.0,5.5,New England Patriots,-103.0,-5.5
9,3195,-4.0,-30,Pittsburgh Steelers,-107.0,5.5,New England Patriots,-103.0,-5.5


In [707]:
def determinePick(s):
    if  'Bet on home' in s:
        return 'H'
    elif 'Bet on away' in s:
        return 'A'
    else:
        return 'N'


past_pred.loc[(past_pred.Predicted_Spread <= 0) & \
                 (past_pred.Home_Spread < 0) & \
                 (past_pred.Predicted_Spread < past_pred.Home_Spread ) , 'Bet_choice'] =  "Home Favorite More -- Bet on home"

#Home Favorite Less -- Bet on away #50%
past_pred.loc[(past_pred.Predicted_Spread <= 0) & \
                 (past_pred.Home_Spread < 0) & \
                 (past_pred.Predicted_Spread > past_pred.Home_Spread), 'Bet_choice'] = "Home Favorite Less -- Bet on away"

# Home Favorite Dog -- Bet on home #63.64%
past_pred.loc[(past_pred.Predicted_Spread <= 0) & \
                 (past_pred.Home_Spread > 0)  , 'Bet_choice'] = "Home Favorite Dog -- Bet on home"

# Home Dog More -- Bet on away #no bets
past_pred.loc[(past_pred.Predicted_Spread >= 0) & \
                 (past_pred.Home_Spread > 0) & \
                 (past_pred.Predicted_Spread > past_pred.Home_Spread ) , 'Bet_choice'] = "Home Dog More -- Bet on away"

# Home Dog Less -- Bet on home #100%, 2 games
past_pred.loc[(past_pred.Predicted_Spread >= 0) & \
                 (past_pred.Home_Spread > 0) & \
                 (past_pred.Predicted_Spread < past_pred.Home_Spread) , 'Bet_choice'] = "Home Dog Less -- Bet on home"

# Home Dog Favorite -- Bet on away #100%, 9 games
past_pred.loc[(past_pred.Predicted_Spread >= 0) & \
                 (past_pred.Home_Spread < 0) , 'Bet_choice'] = "Home Dog Favorite -- Bet on away"

past_pred.loc[(past_pred.Predicted_Spread == past_pred.Home_Spread),'Bet_choice'] = 'Equal Spread'


past_pred['Bet'] = past_pred['Bet_choice'].map(determinePick)
past_pred.groupby('GameNum')['Bet_choice'].value_counts().unstack().T

GameNum,185,186,187,188,189,190,191,192,193,194,...,3670,3671,3672,3673,3674,3675,3676,3677,3678,3679
Bet_choice,,,,,,,,,,,,,,,,,,,,,
Equal Spread,26.0,NaN,19.0,NaN,21.0,NaN,26.0,NaN,16.0,NaN,...,NaN,NaN,9.0,19.0,6.0,19.0,15.0,NaN,NaN,NaN
Home Dog Favorite -- Bet on away,NaN,NaN,NaN,56.0,NaN,NaN,11.0,NaN,NaN,1.0,...,21.0,31.0,86.0,NaN,5.0,NaN,22.0,4.0,94.0,NaN
Home Dog Less -- Bet on home,52.0,12.0,80.0,NaN,NaN,87.0,NaN,46.0,123.0,NaN,...,NaN,NaN,NaN,75.0,NaN,20.0,NaN,NaN,NaN,77.0
Home Dog More -- Bet on away,27.0,NaN,38.0,NaN,NaN,35.0,NaN,15.0,23.0,NaN,...,NaN,NaN,NaN,34.0,NaN,105.0,NaN,NaN,NaN,37.0
Home Favorite Dog -- Bet on home,64.0,173.0,39.0,NaN,NaN,43.0,NaN,104.0,4.0,NaN,...,NaN,NaN,NaN,19.0,NaN,10.0,NaN,NaN,NaN,38.0
Home Favorite Less -- Bet on away,NaN,NaN,NaN,112.0,98.0,NaN,32.0,NaN,NaN,8.0,...,151.0,147.0,42.0,NaN,7.0,NaN,46.0,26.0,56.0,NaN
Home Favorite More -- Bet on home,NaN,NaN,NaN,5.0,21.0,NaN,102.0,NaN,NaN,145.0,...,NaN,NaN,20.0,NaN,161.0,NaN,73.0,121.0,NaN,NaN


In [708]:
no_bet_games = past_pred[past_pred['Bet']=='N']
no_bet_games

,GameNum,Predicted_Spread,Game_Spread,Away,Away_Payout,Away_Spread,Home,Home_Payout,Home_Spread,Bet_choice,Bet
179,2944,2.0,10,New England Patriots,-107.0,-2.0,Los Angeles Rams,-103.0,2.0,Equal Spread,N
183,2944,2.0,10,New England Patriots,-107.0,-2.0,Los Angeles Rams,-103.0,2.0,Equal Spread,N
187,2944,2.0,10,New England Patriots,-107.0,-2.0,Los Angeles Rams,-103.0,2.0,Equal Spread,N
191,2944,2.0,10,New England Patriots,-107.0,-2.0,Los Angeles Rams,-103.0,2.0,Equal Spread,N
222,2944,2.0,10,New England Patriots,-107.0,-2.0,Los Angeles Rams,-103.0,2.0,Equal Spread,N
233,2944,2.0,10,New England Patriots,-107.0,-2.0,Los Angeles Rams,-103.0,2.0,Equal Spread,N
239,2944,2.0,10,New England Patriots,-107.0,-2.0,Los Angeles Rams,-103.0,2.0,Equal Spread,N
253,2944,2.0,10,New England Patriots,-107.0,-2.0,Los Angeles Rams,-103.0,2.0,Equal Spread,N
264,2944,2.0,10,New England Patriots,-107.0,-2.0,Los Angeles Rams,-103.0,2.0,Equal Spread,N
273,2944,2.0,10,New England Patriots,-107.0,-2.0,Los Angeles Rams,-103.0,2.0,Equal Spread,N


In [709]:
confidence = past_pred.groupby('GameNum')['Bet'].value_counts().unstack().T.max() / past_pred.groupby('GameNum')['Bet'].value_counts().unstack().T.sum()

unique_games = past_pred.drop_duplicates(subset='GameNum').set_index('GameNum')

unique_games['Bet'] = past_pred.groupby(['GameNum'])['Bet'].value_counts().unstack().fillna(0).T.idxmax()
unique_games['Bet_choice'] = past_pred.groupby(['GameNum'])['Bet_choice'].value_counts().unstack().fillna(0).T.idxmax()

unique_games['Confidence'] = confidence

In [710]:
# unique_games.loc[(unique_games.Home_Spread < unique_games.Game_Spread) & \
#                      (unique_games.Bet == "A")  , 'Win'] = 1

# unique_games.loc[(unique_games.Home_Spread > unique_games.Game_Spread) & \
#                      (unique_games.Bet == "H")  , 'Win'] = 1

# unique_games.loc[(unique_games.Home_Spread > unique_games.Game_Spread) & \
#                      (unique_games.Bet == "A")  , 'Win'] = 0

# unique_games.loc[(unique_games.Home_Spread < unique_games.Game_Spread) & \
#                      (unique_games.Bet == "H")  , 'Win'] = 0

unique_games.loc[(unique_games.Home_Spread < unique_games.Game_Spread) & \
                     (unique_games.Bet == "A")  , 'Win'] = 'Win'

unique_games.loc[(unique_games.Home_Spread > unique_games.Game_Spread) & \
                     (unique_games.Bet == "H")  , 'Win'] = 'Win'

unique_games.loc[(unique_games.Home_Spread > unique_games.Game_Spread) & \
                     (unique_games.Bet == "A")  , 'Win'] = 'Lose'

unique_games.loc[(unique_games.Home_Spread < unique_games.Game_Spread) & \
                     (unique_games.Bet == "H")  , 'Win'] = 'Lose'

unique_games.loc[(unique_games.Bet == "N")  , 'Win'] = 'No_bet'

unique_games.loc[(unique_games.Home_Spread == unique_games.Game_Spread)  , 'Win'] = 'Push'


unique_games.groupby('Bet_choice')['Win'].value_counts()
#unique_games[unique_games['Confidence'] <= 1][unique_games['Confidence'] >= .].groupby('Bet')['Win'].value_counts().unstack().sum()

Bet_choice                         Win 
Home Dog Favorite -- Bet on away   Win      47
                                   Lose     39
                                   Push      7
Home Dog Less -- Bet on home       Lose     55
                                   Win      53
                                   Push      5
Home Dog More -- Bet on away       Lose      9
                                   Win       9
Home Favorite Dog -- Bet on home   Lose     62
                                   Win      62
                                   Push      4
Home Favorite Less -- Bet on away  Win     143
                                   Lose    121
                                   Push      5
Home Favorite More -- Bet on home  Win      47
                                   Lose     37
                                   Push      5
Name: Win, dtype: int64

Winrate 53.0%

Away Winrate 54.8%

Home Winrate 50.8%


In [711]:
unique_games['Win'].value_counts()

Win     361
Lose    323
Push     26
Name: Win, dtype: int64

In [712]:
unique_games.groupby('Bet')['Win'].value_counts()

Bet  Win 
A    Win     205
     Lose    171
     Push     12
H    Win     156
     Lose    152
     Push     14
Name: Win, dtype: int64

High Confidence is associated with a higher winrate for Away games (57.3%)

High Confidence is associated with a lower winrate for Home games

High Confidence > 90%

Away Confidence = 1,  59.6%


High Sample size leads to more trust in confidence

Away Confidence = 1, 62.6%

Away Confidence > .9

In [734]:
bet_home_games = unique_games[unique_games['Bet']=='H']

print(bet_home_games[bet_home_games['Confidence'] > .9].groupby('Bet')['Win'].value_counts().unstack().sum())

bet_away_games = unique_games[unique_games['Bet']=='A']

print(bet_away_games[bet_away_games['Confidence'] == 1].groupby('Bet')['Win'].value_counts().unstack().sum())

Win
Lose    87
Push     6
Win     86
dtype: int64
Win
Lose    31
Push     2
Win     52
dtype: int64


In [714]:
from scipy import stats

gamedata = past_pred.groupby('GameNum')['Predicted_Spread'].agg({'Mean':np.mean,'Max':np.max,'Min':np.min,'StdDev':np.std,'Mode':stats.mode,'Median':np.median}).reset_index()
gamedata = gamedata.merge(unique_games[['Away',
 'Home',
 'Home_Spread',
 'Home_Payout',
 'Bet_choice',
 'Confidence',
 'Bet',
 'Win']],left_on='GameNum',right_on='GameNum')
gamedata[gamedata['Confidence']==1]['Win'].value_counts()

C:\Users\chris\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


Win     83
Lose    65
Push     5
Name: Win, dtype: int64

# This shows that you want to choose differently for stdDev based on home or away bet

If betting HOME, you want the mean to be more than 1 stdDev of the Vegas Line

    45% within 1 stdDev 
    vs 
    51% outside 1 stdDev

If betting AWAY, you want the mean to be more than 1 stdDev from Vegas Line

    50% within 1 stdDev
    vs
    53.7% outside 1 stdDev

In [715]:
gamedata.loc[(gamedata.Mean > gamedata.Home_Spread), 'one_stdDev'] =  gamedata.Mean-gamedata.StdDev
gamedata.loc[(gamedata.Mean < gamedata.Home_Spread), 'one_stdDev'] =  gamedata.Mean+gamedata.StdDev

gamedata.loc[(gamedata.Home_Spread < 0) & \
             (gamedata.Mean > gamedata.Home_Spread) & \
             (gamedata.one_stdDev > gamedata.Home_Spread), 'within_one_stdDev'] =  0
gamedata.loc[(gamedata.Home_Spread < 0) & \
             (gamedata.Mean < gamedata.Home_Spread) & \
             (gamedata.one_stdDev < gamedata.Home_Spread), 'within_one_stdDev'] =  0
gamedata.loc[(gamedata.Home_Spread > 0) & \
             (gamedata.Mean > gamedata.Home_Spread) & \
             (gamedata.one_stdDev > gamedata.Home_Spread), 'within_one_stdDev'] =  0
gamedata.loc[(gamedata.Home_Spread > 0) & \
             (gamedata.Mean < gamedata.Home_Spread) & \
             (gamedata.one_stdDev < gamedata.Home_Spread), 'within_one_stdDev'] =  0

gamedata.loc[(gamedata.Home_Spread < 0) & \
             (gamedata.Mean > gamedata.Home_Spread) & \
             (gamedata.one_stdDev < gamedata.Home_Spread), 'within_one_stdDev'] =  1
gamedata.loc[(gamedata.Home_Spread < 0) & \
             (gamedata.Mean < gamedata.Home_Spread) & \
             (gamedata.one_stdDev > gamedata.Home_Spread), 'within_one_stdDev'] =  1
gamedata.loc[(gamedata.Home_Spread > 0) & \
             (gamedata.Mean > gamedata.Home_Spread) & \
             (gamedata.one_stdDev < gamedata.Home_Spread), 'within_one_stdDev'] =  1
gamedata.loc[(gamedata.Home_Spread > 0) & \
             (gamedata.Mean < gamedata.Home_Spread) & \
             (gamedata.one_stdDev > gamedata.Home_Spread), 'within_one_stdDev'] =  1

In [716]:
gamedata.loc[(gamedata.Mean > gamedata.Home_Spread), 'two_stdDev'] =  gamedata.Mean-(2*gamedata.StdDev)
gamedata.loc[(gamedata.Mean < gamedata.Home_Spread), 'two_stdDev'] =  gamedata.Mean+(2*gamedata.StdDev)

gamedata.loc[(gamedata.Home_Spread < 0) & \
             (gamedata.Mean > gamedata.Home_Spread) & \
             (gamedata.two_stdDev > gamedata.Home_Spread), 'within_two_stdDev'] =  0
gamedata.loc[(gamedata.Home_Spread < 0) & \
             (gamedata.Mean < gamedata.Home_Spread) & \
             (gamedata.two_stdDev < gamedata.Home_Spread), 'within_two_stdDev'] =  0
gamedata.loc[(gamedata.Home_Spread > 0) & \
             (gamedata.Mean > gamedata.Home_Spread) & \
             (gamedata.two_stdDev > gamedata.Home_Spread), 'within_two_stdDev'] =  0
gamedata.loc[(gamedata.Home_Spread > 0) & \
             (gamedata.Mean < gamedata.Home_Spread) & \
             (gamedata.two_stdDev < gamedata.Home_Spread), 'within_two_stdDev'] =  0

gamedata.loc[(gamedata.Home_Spread < 0) & \
             (gamedata.Mean > gamedata.Home_Spread) & \
             (gamedata.two_stdDev < gamedata.Home_Spread), 'within_two_stdDev'] =  1
gamedata.loc[(gamedata.Home_Spread < 0) & \
             (gamedata.Mean < gamedata.Home_Spread) & \
             (gamedata.two_stdDev > gamedata.Home_Spread), 'within_two_stdDev'] =  1
gamedata.loc[(gamedata.Home_Spread > 0) & \
             (gamedata.Mean > gamedata.Home_Spread) & \
             (gamedata.two_stdDev < gamedata.Home_Spread), 'within_two_stdDev'] =  1
gamedata.loc[(gamedata.Home_Spread > 0) & \
             (gamedata.Mean < gamedata.Home_Spread) & \
             (gamedata.two_stdDev > gamedata.Home_Spread), 'within_two_stdDev'] =  1


print 'Away\n: ',gamedata[gamedata['Bet']=='A'].groupby('within_one_stdDev')['Win'].value_counts(),'\n'

print 'Home\n: ',gamedata[gamedata['Bet']=='H'].groupby('within_one_stdDev')['Win'].value_counts(),'\n'

print 'Away\n: ',gamedata[gamedata['Bet']=='A'].groupby('within_two_stdDev')['Win'].value_counts(),'\n'

print 'Home\n: ',gamedata[gamedata['Bet']=='H'].groupby('within_two_stdDev')['Win'].value_counts(),'\n'

Away
:  within_one_stdDev  Win 
0.0                Win     148
                   Lose    114
                   Push      6
1.0                Lose     57
                   Win      57
                   Push      6
Name: Win, dtype: int64 

Home
:  within_one_stdDev  Win 
0.0                Lose    100
                   Win      99
                   Push      7
1.0                Win      57
                   Lose     52
                   Push      7
Name: Win, dtype: int64 

Away
:  within_two_stdDev  Win 
0.0                Win      92
                   Lose     72
                   Push      2
1.0                Win     113
                   Lose     99
                   Push     10
Name: Win, dtype: int64 

Home
:  within_two_stdDev  Win 
0.0                Lose    66
                   Win     58
                   Push     5
1.0                Win     98
                   Lose    86
                   Push     9
Name: Win, dtype: int64 



Weekly Winrate based on only betting away = 57.0%

Weekly Winrate based on only betting home = 52.8%

***Both include weeks where no bets were made***


Weekly Winrate based on Away + 90% confidence = 57.6%

Weekly Winrate based on Home + 90% confidence = 55.8%

In [717]:
weekly_data = gamedata.merge(all_games[all_games['H/A']=='H'][['Season','Week','GameNum']], left_on='GameNum', right_on='GameNum')
mask = weekly_data['Bet']=='H'
weekly_winrate = weekly_data[mask].groupby(['Season','Week'])['Win'].value_counts().unstack()['Win'] / weekly_data[mask].groupby(['Season','Week'])['Win'].value_counts().unstack().T.sum()
np.average(list(weekly_winrate.dropna()))
weely_winrate = weekly_data[mask].groupby(['Season','Week'])['Win'].value_counts().unstack()['Win'].fillna(0) / weekly_data[mask].groupby(['Season','Week'])['Win'].value_counts().unstack()['Lose'].fillna(0)
np.average(list(weekly_winrate.dropna()))

0.5108370943477326

In [718]:
weekly_data = gamedata.merge(all_games[all_games['H/A']=='H'][['Season','Week','GameNum']], left_on='GameNum', right_on='GameNum')
mask = (weekly_data['Bet']=='A')
weekly_winrate = weekly_data[mask].groupby(['Season','Week'])['Win'].value_counts().unstack()['Win'] / weekly_data[mask].groupby(['Season','Week'])['Win'].value_counts().unstack().T.sum()
np.average(list(weekly_winrate.dropna()))

0.5520260850260851

In [719]:
weekly_data = gamedata.merge(all_games[all_games['H/A']=='H'][['Season','Week','GameNum']], left_on='GameNum', right_on='GameNum')
mask = (weekly_data['Bet']=='H') & (weekly_data['Confidence'] == 1)
weekly_winrate = weekly_data[mask].groupby(['Season','Week'])['Win'].value_counts().unstack()['Win'] / weekly_data[mask].groupby(['Season','Week'])['Win'].value_counts().unstack().T.sum()
np.average(list(weekly_winrate.dropna()))

0.6572463768115941

In [720]:
weekly_data = gamedata.merge(all_games[all_games['H/A']=='H'][['Season','Week','GameNum']], left_on='GameNum', right_on='GameNum')
mask = (weekly_data['Bet']=='A') & (weekly_data['Confidence'] == 1)
weekly_winrate = weekly_data[mask].groupby(['Season','Week'])['Win'].value_counts().unstack()['Win'] / weekly_data[mask].groupby(['Season','Week'])['Win'].value_counts().unstack().T.sum()
np.average(list(weekly_winrate.dropna()))

0.753225806451613

In [721]:
mask = (weekly_data['Bet']=='A') & (weekly_data['Home_Spread'] > 0)
weekly_data[mask]['Win'].value_counts()

Lose    6
Win     6
Name: Win, dtype: int64

Vegas is consistently wrong on the away side for 2019
    
    57% away wins on spread when home is favored
    52% away wins on spread when away is favored
    
    43% home wins on spread when home is favored
    48% home wins on spread when away is favored

Vegas slightly favors away over 2017, 2018, 2019

    52.5% away wins on spread when home is favored
    51.0% away wins on spread when away is favored
    
    47.5% home wins on spread when home is favored
    49.0% home wins on spread when away is favored

In [722]:
test = lines.merge(all_games[all_games['H/A']=='H'][['GameNum','PF','PA']])
test['Game_Spread'] = test['PA'].astype(int) - test['PF'].astype(int)

mask = test['Home_Spread'] < 0

# test = test.merge(unique_games)

test[mask][test[mask]['Game_Spread'] > test[mask]['Home_Spread']].count()[0] * 1.0 / test[mask].count()[0]

test.merge(unique_games, right_on='GameNum', left_on='GameNum')[['Away_x',
 'Home_x',
 'Home_Spread_x',
 'GameNum',
 'PF',
 'PA',
 'Game_Spread_x',
 'Game_Spread_y',
 'Predicted_Spread',
 'Away_y',
 'Home_y',
 'Home_Spread_y',
 'Bet_choice',
 'Bet',
 'Confidence',
 'Win']]

,Away_x,Home_x,Home_Spread_x,GameNum,PF,PA,Game_Spread_x,Game_Spread_y,Predicted_Spread,Away_y,Home_y,Home_Spread_y,Bet_choice,Bet,Confidence,Win
0,Kansas City Chiefs,New England Patriots,-9.0,2710,27,42,15,15,-11.0,Kansas City Chiefs,New England Patriots,-9.0,Home Favorite Less -- Bet on away,A,0.700637,Win
1,Oakland Raiders,Tennessee Titans,-2.5,3425,16,26,10,10,-2.0,Oakland Raiders,Tennessee Titans,-2.5,Home Favorite More -- Bet on home,H,0.672956,Lose
2,Arizona Cardinals,Detroit Lions,2.0,185,35,23,-12,-12,1.0,Arizona Cardinals,Detroit Lions,2.0,Home Favorite Dog -- Bet on home,H,0.686391,Win
3,Atlanta Falcons,Chicago Bears,6.5,401,17,23,6,6,2.0,Atlanta Falcons,Chicago Bears,6.5,Home Dog Less -- Bet on home,H,0.725714,Win
4,Pittsburgh Steelers,Cleveland Browns,10.0,1531,18,21,3,3,8.0,Pittsburgh Steelers,Cleveland Browns,10.0,Home Dog Less -- Bet on home,H,0.913907,Win
5,Jacksonville Jaguars,Houston Texans,-6.0,2334,7,29,22,22,-3.0,Jacksonville Jaguars,Houston Texans,-6.0,Home Favorite Less -- Bet on away,A,0.836735,Win
6,New York Jets,Buffalo Bills,-7.5,831,21,12,-9,-9,-2.0,New York Jets,Buffalo Bills,-7.5,Home Dog Favorite -- Bet on away,A,1.000000,Lose
7,Baltimore Ravens,Cincinnati Bengals,-2.5,627,0,20,20,20,0.0,Baltimore Ravens,Cincinnati Bengals,-2.5,Home Favorite Less -- Bet on away,A,0.753086,Win
8,Philadelphia Eagles,Washington Redskins,-3.0,3477,17,30,13,13,2.0,Philadelphia Eagles,Washington Redskins,-3.0,Home Dog Favorite -- Bet on away,A,0.993750,Win
9,Indianapolis Colts,Los Angeles Rams,-3.5,2482,46,9,-37,-37,-7.0,Indianapolis Colts,Los Angeles Rams,-3.5,Home Favorite Less -- Bet on away,A,0.625000,Lose


Broken

In [723]:
mask = unique_games['Home_Spread'] < 0
away_win_home_favor = unique_games[mask][unique_games[mask]['Game_Spread'] > unique_games[mask]['Home_Spread']].count()[0]
print away_win_home_favor * 1.0 / unique_games[mask].count()[0]

mask = unique_games['Home_Spread'] < 0
home_win_home_favor = unique_games[mask][unique_games[mask]['Game_Spread'] < unique_games[mask]['Home_Spread']].count()[0]
print home_win_home_favor * 1.0 / unique_games[mask].count()[0]

mask = unique_games['Home_Spread'] > 0
away_win_away_favor = unique_games[mask][unique_games[mask]['Game_Spread'] > unique_games[mask]['Home_Spread']].count()[0]
print away_win_away_favor * 1.0 / unique_games[mask].count()[0]

mask = unique_games['Home_Spread'] > 0
home_win_away_favor = unique_games[mask][unique_games[mask]['Game_Spread'] < unique_games[mask]['Home_Spread']].count()[0]
print home_win_away_favor * 1.0 / unique_games[mask].count()[0]

unique_games[mask].count()

0.5121951219512195
0.4501108647450111
0.4864864864864865
0.47876447876447875


Predicted_Spread    259
Game_Spread         259
Away                259
Away_Payout         259
Away_Spread         259
Home                259
Home_Payout         259
Home_Spread         259
Bet_choice          259
Bet                 259
Confidence          259
Win                 259
dtype: int64

In [724]:
mask = weekly_data['Home_Spread'] < 0
mask2 = weekly_data['Bet'] == 'A'
weekly_data[mask2][mask]['Win'].value_counts()

C:\Users\chris\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Win     199
Lose    165
Push     12
Name: Win, dtype: int64

Win rate by team not statistically significant 

Highest winrate Eagles with 80%

In [725]:
unique_games.groupby('Home')['Win'].value_counts().unstack().fillna(0) + unique_games.groupby('Away')['Win'].value_counts().unstack().fillna(0)

Win,Lose,Push,Win
Home,,,
Arizona Cardinals,18.0,1.0,24.0
Atlanta Falcons,17.0,0.0,26.0
Baltimore Ravens,16.0,2.0,27.0
Buffalo Bills,22.0,2.0,19.0
Carolina Panthers,18.0,0.0,27.0
Chicago Bears,20.0,1.0,25.0
Cincinnati Bengals,19.0,1.0,24.0
Cleveland Browns,23.0,1.0,20.0
Dallas Cowboys,23.0,3.0,18.0


Mode typically agrees with the most popular bet for the model (95% of the time)

When it disagrees with the most popular bet, take the mode bet instead (wins 7/8 games, 87.5% winrate)


DISPROVED BY ADDITIONAL DATA

In [726]:
testmode = gamedata

def findMode(s):
    return s[0][0]

testmode = testmode.merge(past_pred.drop_duplicates(subset='GameNum')[['GameNum','Game_Spread']],left_on='GameNum',right_on='GameNum')
testmode['Mode_Spread'] = testmode['Mode'].apply(findMode)

testmode.loc[(testmode.Mode_Spread < testmode.Home_Spread), 'Mode_Bet'] = 'H'
testmode.loc[(testmode.Mode_Spread > testmode.Home_Spread), 'Mode_Bet'] = 'A'
testmode.loc[(testmode.Mode_Spread == testmode.Home_Spread), 'Mode_Bet'] = 'N'

testmode.loc[(testmode.Mode_Bet == 'H') & \
             (testmode.Home_Spread > testmode.Game_Spread), 'Mode_Win'] = 1
testmode.loc[(testmode.Mode_Bet == 'A') & \
             (testmode.Home_Spread < testmode.Game_Spread), 'Mode_Win'] = 1
testmode.loc[(testmode.Mode_Bet == 'H') & \
             (testmode.Home_Spread < testmode.Game_Spread), 'Mode_Win'] = 0
testmode.loc[(testmode.Mode_Bet == 'A') & \
             (testmode.Home_Spread > testmode.Game_Spread), 'Mode_Win'] = 0
testmode.loc[(testmode.Mode_Bet == 'N'), 'Mode_Win'] = 'No bet'

mask = testmode['Mode_Bet'] != testmode['Bet']
testmode[mask]['Mode_Win'].value_counts()

No bet    30
0.0       10
1.0        5
Name: Mode_Win, dtype: int64

Median agrees with the most popular bet at all times

If Median_Bet != Mode_Bet, go with the Mode_bet

DISPROVED BY ADDITIONAL DATA

In [727]:
testmedian = gamedata

testmedian = testmedian.merge(past_pred.drop_duplicates(subset='GameNum')[['GameNum','Game_Spread']],left_on='GameNum',right_on='GameNum')

testmedian.loc[(testmedian.Median < testmedian.Home_Spread), 'Median_Bet'] = 'H'
testmedian.loc[(testmedian.Median > testmedian.Home_Spread), 'Median_Bet'] = 'A'
testmedian.loc[(testmedian.Median == testmedian.Home_Spread), 'Median_Bet'] = 'N'

testmedian.loc[(testmedian.Median_Bet == 'H') & \
             (testmedian.Home_Spread > testmedian.Game_Spread), 'Median_Win'] = 1
testmedian.loc[(testmedian.Median_Bet == 'A') & \
             (testmedian.Home_Spread < testmedian.Game_Spread), 'Median_Win'] = 1
testmedian.loc[(testmedian.Median_Bet == 'H') & \
             (testmedian.Home_Spread < testmedian.Game_Spread), 'Median_Win'] = 0
testmedian.loc[(testmedian.Median_Bet == 'A') & \
             (testmedian.Home_Spread > testmedian.Game_Spread), 'Median_Win'] = 0
testmedian.loc[(testmedian.Median_Bet == 'N'), 'Median_Win'] = 'No bet'

mask = testmedian['Median_Bet'] != testmode['Mode_Bet']
testmedian[mask]['Median_Win'].value_counts()

No bet    13
1.0       13
0.0        9
Name: Median_Win, dtype: int64

Ideally you want a MinMax difference less than 10

MinMax over 10 winrate Away = 47.1%

MinMax over 10 winrate Home = 46.5%

In [728]:
mask = ((gamedata['Min'] - gamedata['Max']) *-1 >= 10)# | ((gamedata['Min'] - gamedata['Max']) *-1 > 15)
gamedata[mask].groupby('Bet')['Win'].value_counts()

Bet  Win 
A    Win     169
     Lose    135
     Push     10
H    Win     133
     Lose    126
     Push     12
Name: Win, dtype: int64

In [729]:
gamedata

,GameNum,Min,Max,Median,Mode,StdDev,Mean,Away,Home,Home_Spread,Home_Payout,Bet_choice,Confidence,Bet,Win,one_stdDev,within_one_stdDev,two_stdDev,within_two_stdDev
0,185,-9.0,6.0,0.0,"([1.0], [31])",2.726574,0.017751,Arizona Cardinals,Detroit Lions,2.0,-103.0,Home Favorite Dog -- Bet on home,0.686391,H,Win,2.744326,1.0,5.470900,1.0
1,186,-8.0,4.0,-3.0,"([-3.0], [46])",1.918921,-3.237838,Arizona Cardinals,Indianapolis Colts,6.5,100.0,Home Favorite Dog -- Bet on home,1.000000,H,Win,-1.318917,0.0,0.600003,0.0
2,187,-6.0,10.0,1.0,"([1.0], [33])",2.834115,1.545455,Dallas Cowboys,Arizona Cardinals,3.0,-115.0,Home Dog Less -- Bet on home,0.676136,H,Lose,4.379570,1.0,7.213685,1.0
3,188,-9.0,4.0,-2.0,"([-2.0], [26])",2.487136,-1.705202,San Francisco 49Ers,Arizona Cardinals,-6.5,-105.0,Home Favorite Less -- Bet on away,0.971098,A,Win,-4.192338,0.0,-6.679474,1.0
4,189,-10.0,-1.0,-5.0,"([-5.0], [36])",1.735548,-4.771429,Arizona Cardinals,Philadelphia Eagles,-6.0,-107.0,Home Favorite Less -- Bet on away,0.700000,A,Lose,-6.506977,1.0,-8.242525,1.0
5,190,-5.0,12.0,1.0,"([0.0], [34])",2.610245,0.939394,Tampa Bay Buccaneers,Arizona Cardinals,2.5,102.0,Home Dog Less -- Bet on home,0.787879,H,Win,3.549639,1.0,6.159883,1.0
6,191,-14.0,1.0,-4.0,"([-4.0], [26])",2.751726,-4.356725,Arizona Cardinals,Los Angeles Rams,-3.0,-115.0,Home Favorite More -- Bet on home,0.596491,H,Win,-1.604999,1.0,1.146728,1.0
7,192,-13.0,8.0,-2.0,"([-3.0], [24])",3.050184,-1.406061,Arizona Cardinals,San Francisco 49Ers,2.5,-105.0,Home Favorite Dog -- Bet on home,0.909091,H,Lose,1.644123,0.0,4.694307,1.0
8,193,-2.0,11.0,4.0,"([3.0], [31])",2.456426,3.951807,Seattle Seahawks,Arizona Cardinals,6.0,-105.0,Home Dog Less -- Bet on home,0.765060,H,Push,6.408233,1.0,8.864659,1.0
9,194,-14.0,0.0,-5.5,"([-5.0], [33])",2.403560,-5.883117,Arizona Cardinals,Houston Texans,-2.5,-108.0,Home Favorite More -- Bet on home,0.941558,H,Win,-3.479557,0.0,-1.075998,1.0
